In [ ]:
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path='boston_housing.npz', test_split=0.1, seed=113
)
T = x_train.shape[0]

In [ ]:
import numpy as np
from typing import Callable
from sklearn.metrics import mean_squared_error


class RRDV():
  def __init__(self, kernel: Callable[[np.ndarray, np.ndarray], np.ndarray], a: float):
    self.kernel = kernel
    self.a = a

  # dim y = T
  def optimize(self, X: np.ndarray, y: np.ndarray):
    self.X = X
    self.y = y
    K = self.kernel(X,X)
    I = np.eye(K.shape[0])
    self.optimized = np.linalg.inv((K + self.a * I))

  def predict(self, x: np.ndarray):
    k = self.kernel(self.X, x)
    return  np.matmul(self.y, np.matmul(self.optimized, k))


In [ ]:
def linear_kernel(x1: np.ndarray, x2: np.ndarray):
  return np.dot(x1 , x2.T)

In [ ]:
def poly_kernel(x1: np.ndarray, x2: np.ndarray):
  return (np.dot(x1 , x2.T) + 1)**2

In [ ]:
def gauss_kernel(x1: np.ndarray, x2: np.ndarray, sigma=1.0):
    sq_dists = np.sum(x1**2, axis=1).reshape(-1, 1) + np.sum(x2**2, axis=1) - 2 * np.dot(x1, x2.T)
    return np.exp(-sq_dists / (2 * sigma**2))

In [ ]:
lin = RRDV(linear_kernel, 7)
lin.optimize(x_train, y_train)
y_pred = lin.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 30.105696642090972


In [ ]:
lin = RRDV(poly_kernel, 0.8)
lin.optimize(x_train, y_train)
y_pred = lin.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 20.96472732897727


In [ ]:
lin = RRDV(gauss_kernel, 100)
lin.optimize(x_train, y_train)
y_pred = lin.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 648.4483095428013
